In [3]:
import pyomo
from pyomo.environ import *

In [4]:
from pyomo.core import *
#from pyomo.pysp.annotations import *
#import pyomo.pysp.convert
from pyomo.opt import SolverFactory

In [5]:
import cplex
import itertools

In [6]:
# Ex 1
# Concrete Model

model = ConcreteModel()

model.x = Var([1,2], domain=NonNegativeReals)

model.OBJ = Objective(expr = 2*model.x[1] + 3*model.x[2])

model.Constraint1 = Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)

opt = SolverFactory('cplex',executable='/Applications/CPLEX_Studio128/cplex/bin/x86-64_osx/cplex')
opt.solve(model)

{'Problem': [{'Name': 'tmpspu7s1lq', 'Lower bound': 0.6666666666666666, 'Upper bound': 0.6666666666666666, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 3, 'Number of nonzeros': 3, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Dual simplex - Optimal\\x3a Objective = 6.6666666667e-01', 'Error rc': 0, 'Time': 0.06987524032592773}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [10]:
model.display()

Model unknown

  Variables:
    x : Size=2, Index=x_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          1 :     0 : 0.3333333333333333 :  None : False : False : NonNegativeReals
          2 :     0 :                0.0 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 0.6666666666666666

  Constraints:
    Constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :   1.0 :  1.0 :  None


In [28]:
# Ex 2
# Abstract Model

#from __future__ import division
#import pyomo
#from pyomo.environ import *

model = AbstractModel()

model.m = Param(within=NonNegativeIntegers)
model.n = Param(within=NonNegativeIntegers)

model.I = RangeSet(1, model.m)
model.J = RangeSet(1, model.n)

model.a = Param(model.I, model.J)
model.b = Param(model.I)
model.c = Param(model.J)

# the next line declares a variable indexed by the set J
model.x = Var(model.J, domain=NonNegativeReals)

def obj_expression(model):
    return summation(model.c, model.x)

model.OBJ = Objective(rule=obj_expression)

def ax_constraint_rule(model, i):
    # return the expression for the constraint for i
    return sum(model.a[i,j] * model.x[j] for j in model.J) >= model.b[i]

# the next line creates one constraint for each member of the set model.I
model.AxbConstraint = Constraint(model.I, rule=ax_constraint_rule)


# one way to input the data in AMPL format
# for indexed parameters, the indexes are given before the value

data= {None: {
    'm':{None:1},
    'n':{None:2},
    'a': {(1,1):3, (1,2):4},
    'c': {1:2, 2:3},
    'b': {1: 1} }}
    
instance = model.create_instance(data)
opt = SolverFactory('cplex', executable='/Applications/CPLEX_Studio1262/cplex/bin/x86-64_osx/cplex')
opt.solve(instance)

{'Problem': [{'Name': 'tmpk54ufg6v', 'Lower bound': 0.6666666666666666, 'Upper bound': 0.6666666666666666, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 3, 'Number of nonzeros': 3, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Dual simplex - Optimal\\x3a Objective = 6.6666666667e-01', 'Error rc': 0, 'Time': 0.07142996788024902}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [29]:
instance.display()

Model unknown

  Variables:
    x : Size=2, Index=J
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          1 :     0 : 0.3333333333333333 :  None : False : False : NonNegativeReals
          2 :     0 :                0.0 :  None : False : False : NonNegativeReals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 0.6666666666666666

  Constraints:
    AxbConstraint : Size=1
        Key : Lower : Body : Upper
          1 :   1.0 :  1.0 :  None


In [14]:
#import pyomo
import pyomo.environ as pyo
#from pyomo.opt import SolverFactory

In [15]:
# iterative1.py
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Create a solver
opt = pyo.SolverFactory('glpk')

#
# A simple model with binary variables and
# an empty constraint list.
#
model = pyo.AbstractModel()
model.n = pyo.Param(default=4)
model.x = pyo.Var(pyo.RangeSet(model.n), within=pyo.Binary)
def o_rule(model):
    return pyo.summation(model.x)
model.o = pyo.Objective(rule=o_rule)
model.c = pyo.ConstraintList()

# Create a model instance and optimize
instance = model.create_instance()
results = opt.solve(instance)
instance.display()

# Iterate to eliminate the previously found solution
for i in range(5):
    expr = 0
    for j in instance.x:
        if pyo.value(instance.x[j]) == 0:
            expr += instance.x[j]
        else:
            expr += (1-instance.x[j])
    instance.c.add( expr >= 1 )
    results = opt.solve(instance)
    print ("\n===== iteration",i)
    instance.display()

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    o : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   0.0

  Constraints:
    c : Size=0
        Key : Lower : Body : Upper

===== iteration 0
Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    o : Size=1, Index=None, Active=True
        Key  : Acti